### 🧠 Full Pipeline: Network Analysis + Visualization

This script performs both:

1. **Topological analysis**: centrality metrics, Louvain modularity, efficiency, etc.
2. **Visualization**: spring-layout rendering with module color annotation

#### 📥 Input

- Network edge list CSV (`*_edges.csv`) for each group, containing:
  - Node1, Node2, Weight

#### 📤 Output

- `[group]_network_analysis.csv`: tabular summary of node-level topology
- `[group]_network.png`: annotated visualization with color-coded modules

#### 📊 Metrics Included

- Module (Louvain)
- Degree Centrality
- Betweenness Centrality
- Eigenvector Centrality

#### 🎨 Visualization Features

- Spring-layout with edge bundling
- Node size = weighted degree
- Node color = module
- Edge color = intra/inter module

#### 🔁 Extension Ideas

- Compare module structures across groups
- Rank hubs by centrality
- Overlay phenotype metadata

> All output images saved to: `/group_network_visualizations`


In [ ]:
import os
import networkx as nx
import pandas as pd
import numpy as np
import community.community_louvain as community_louvain
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.lines import Line2D

# Input and output directories
input_dir = r'..group results'  # Replace with your network edge list directory
output_dir = r'..group results'  # Replace with your desired output directory
output_images_dir = os.path.join(output_dir, 'group_network_visualizations')
os.makedirs(output_dir, exist_ok=True)
os.makedirs(output_images_dir, exist_ok=True)

# Find all *_edges.csv files
network_files = [f for f in os.listdir(input_dir) if f.endswith('_edges.csv')]

# Function to visualize network with community modules
def draw_group_network(G, group_name, partition, pos, output_path):
    unique_modules = set(partition.values())
    color_palette = cm.get_cmap('plasma')
    colors = color_palette(np.linspace(0.3, 1, len(unique_modules)))  # Avoid very light colors
    module_colors = {
        module: color_palette(i / (len(unique_modules) - 1) * 0.7 + 0.3)
        for i, module in enumerate(sorted(unique_modules))
    }
    node_colors = [module_colors[partition[node]] for node in G.nodes()]
    node_sizes = [G.degree(node, weight='weight') * 3 for node in G.nodes()]
    edge_colors = [
        module_colors[partition[u]] if partition[u] == partition[v] else "black"
        for u, v in G.edges()
    ]
    edge_widths = [0.02 for _ in G.edges()]

    plt.figure(figsize=(15, 15))
    ax = plt.gca()
    nx.draw_networkx_nodes(G, pos, node_size=node_sizes, node_color=node_colors, alpha=0.9, ax=ax)
    nx.draw_networkx_edges(G, pos, width=edge_widths, edge_color=edge_colors, alpha=0.7, ax=ax)

    # Legend
    legend_elements = [
        Line2D([0], [0], marker='o', color='w', label=f'Module {module}',
               markerfacecolor=module_colors[module], markersize=10)
        for module in unique_modules
    ]
    plt.legend(handles=legend_elements, loc='upper right', title='Modules')
    plt.title(f"Network Visualization: {group_name}", fontsize=20)
    plt.axis('off')
    plt.savefig(output_path, format='png', dpi=300, bbox_inches='tight')
    plt.close()

# Loop over each group's edge file
for edges_file in network_files:
    group_name = edges_file.split('_edges.csv')[0]
    edges_file_path = os.path.join(input_dir, edges_file)
    output_file_path = os.path.join(output_dir, f'{group_name}_network_analysis.csv')
    output_image_path = os.path.join(output_images_dir, f'{group_name}_network.png')

    # Build network graph
    edges_df = pd.read_csv(edges_file_path)
    G = nx.Graph()
    for _, row in edges_df.iterrows():
        G.add_edge(row['Node1'], row['Node2'], weight=row['Weight'])

    # Fix negative weights
    for u, v, data in G.edges(data=True):
        if data['weight'] < 0:
            data['weight'] = abs(data['weight'])

    # Louvain community detection
    partition = community_louvain.best_partition(G, weight='weight')
    nx.set_node_attributes(G, partition, 'Module')

    # Centrality calculations
    degree_centrality = nx.degree_centrality(G)
    betweenness_centrality = nx.betweenness_centrality(G, weight='weight')
    eigenvector_centrality = nx.eigenvector_centrality(G, weight='weight', max_iter=1000)

    # Network robustness metrics
    efficiency = nx.global_efficiency(G)
    avg_degree = sum(dict(G.degree(weight='weight')).values()) / len(G.nodes())

    # Build summary DataFrame
    data = []
    for node in G.nodes():
        data.append({
            "Node": node,
            "Module": partition[node],
            "Degree Centrality": degree_centrality[node],
            "Betweenness Centrality": betweenness_centrality[node],
            "Eigenvector Centrality": eigenvector_centrality[node]
        })
    df = pd.DataFrame(data)
    df.to_csv(output_file_path, index=False)

    # Network layout and visualization
    pos = nx.fruchterman_reingold_layout(G, seed=42, k=5, iterations=500, scale=7)
    draw_group_network(G, group_name, partition, pos, output_image_path)

    print(f"{group_name}: analysis and visualization saved to: {output_file_path} and {output_image_path}")

print("All group networks have been analyzed and visualized.")
